<a href="https://colab.research.google.com/github/revanthjavvaji/SE20UARI071_SE20UARI036_SKU/blob/main/SKU_Assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split


data = pd.read_csv("/content/train.csv")

In [27]:
data

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,0
1,1,1,2010-02-12,46039.49,1
2,1,1,2010-02-19,41595.55,0
3,1,1,2010-02-26,19403.54,0
4,1,1,2010-03-05,21827.90,0
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,0
421566,45,98,2012-10-05,628.10,0
421567,45,98,2012-10-12,1061.02,0
421568,45,98,2012-10-19,760.01,0


In [2]:
data = data.fillna(0)

In [3]:
data['IsHoliday'] = data['IsHoliday'].astype(int)

In [4]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# ARIMA Model

In [5]:
from statsmodels.tsa.arima.model import ARIMA


def train_arima_model(train_data, store, dept):
    store_dept_data = train_data[(train_data['Store'] == store) & (train_data['Dept'] == dept)]
    model = ARIMA(store_dept_data['Weekly_Sales'], order=(5,1,0))
    model_fit = model.fit()
    return model_fit

In [6]:
model_fit = train_arima_model(train_data, store=1, dept=1)

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [7]:
def evaluate_arima_model(model, test_data):
    predictions = model.forecast(steps=12)
    mae = abs(predictions - test_data['Weekly_Sales'][:12]).mean()
    return mae, predictions

In [8]:
mae, predictions = evaluate_arima_model(model_fit, test_data[(test_data['Store'] == 1) & (test_data['Dept'] == 1)])

print(f"Mean Absolute Error: {mae}")
print("Sales Forecast for the next 12 weeks:")
print(predictions)

Mean Absolute Error: 3437.2464228718995
Sales Forecast for the next 12 weeks:
117    21313.965090
118    21457.752290
119    18754.755404
120    19434.224273
121    19750.003843
122    19930.994960
123    20191.147707
124    19622.237142
125    19637.387920
126    19864.920348
127    19825.748503
128    19860.946472
Name: predicted_mean, dtype: float64


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


##Exponential Smoothing (ETS) Forecasting

In [9]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

def train_ets_model(train_data, store, dept):
    store_dept_data = train_data[(train_data['Store'] == store) & (train_data['Dept'] == dept)]
    model = ExponentialSmoothing(store_dept_data['Weekly_Sales'], seasonal='add', seasonal_periods=52, trend='mul')
    model_fit = model.fit()

    return model_fit

In [10]:
model_fit_ets = train_ets_model(train_data, store=1, dept=1)

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


In [11]:
def evaluate_ets_model(model, test_data):
    predictions = model.forecast(steps=12)


    mae = abs(predictions - test_data['Weekly_Sales'][:12]).mean()
    return mae, predictions

In [12]:
mae_ets, predictions_ets = evaluate_ets_model(model_fit_ets, test_data[(test_data['Store'] == 1) & (test_data['Dept'] == 1)])

print(f"Mean Absolute Error (ETS): {mae_ets}")
print("Sales Forecast for the next 12 weeks (ETS):")
print(predictions_ets)

Mean Absolute Error (ETS): 2402.115663449076
Sales Forecast for the next 12 weeks (ETS):
117    16602.631468
118    12322.990615
119    14337.869271
120    17933.349845
121    14385.704090
122    15937.911422
123    13183.544069
124    18858.505226
125    40885.936819
126    22606.909732
127    26347.839915
128    18634.322527
dtype: float64


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
